# LLM-Based D&D Simulation Tutorial

This notebook demonstrates how to use the LLM-based D&D simulation system to create AI-driven gameplay sessions based on real human campaign data.

## Overview

The simulation system includes:
- **Campaign Parameter Extraction**: Analyze human campaigns to extract initialization parameters
- **Character Creation**: Generate D&D characters with personalities and classes
- **Turn Sampling**: Create realistic turn sequences using various sampling methods
- **Character Agents**: LLM-powered characters with memory and decision-making
- **Game Session Management**: Orchestrate complete D&D scenarios

## Setup and Imports

In [1]:
import os
import sys
import json
import random
from pathlib import Path
import textwrap
import numpy as np
import datetime
import os, sys
from collections import Counter

# Import our simulation system
import llm_scaffolding.dnd_simulation as sim

# Set random seed for reproducible results
random.seed(42)


✅ Anthropic API key loaded
✅ OpenAI API key loaded
✅ Google API key loaded


## Model Selection

The simulation system supports multiple LLM providers through liteLLM. You can use:

- **Anthropic Claude**: `claude-3-5-sonnet-20240620`, `claude-3-haiku-20240307`
- **OpenAI GPT**: `gpt-4o`, `gpt-4o-mini`, `gpt-3.5-turbo`
- **Google Gemini**: `gemini-pro`, `gemini-pro-vision`

Make sure your API keys are configured in `api_key.txt` in the project root.

## Step 1: Extract Campaign Parameters

Let's load a real human campaign and extract parameters that we can use to initialize our simulation.

In [2]:
# Choose a campaign file to analyze
campaign_name = '10391-guardians-of-gridori'
campaign_file = os.path.join(os.getcwd(),"..", "data","raw-human-games", "individual_campaigns", campaign_name + '.json')

# You can specify different models for different providers:
# model = "claude-3-5-sonnet-20240620"  # Anthropic Claude Sonnet (default)
# model = "claude-3-haiku-20240307"     # Anthropic Claude haiku
# model = "gpt-4o", "gpt-4o-mini"       # OpenAI GPT-4
# model = "gemini-pro"                  # Google Gemini
model = "claude-3-5-sonnet-20240620"

# Extract parameters from the human campaign
print("📊 Extracting campaign parameters...")
campaign_params = sim.extract_campaign_parameters(campaign_file, model=model)

print(f"\n🎯 Campaign Parameters:")
print(f"  Campaign Name: {campaign_params['campaign_name']}")
print(f"  Total Messages: {campaign_params['total_messages']:,}")
print(f"  Number of Players: {campaign_params['num_players']}")
print(f"  Character Names: {campaign_params['character_names']}")
print(f"  Player Names: {campaign_params['player_names']}")
print(f"  Character Classes: {campaign_params['character_classes']}")
print(f"  Character Genders: {campaign_params['character_genders']}")
print(f"  Character Races: {campaign_params['character_races']}")
print(f"  Initial Scenario: {campaign_params['initial_scenario']}")

print('Character Personalities + Sheets')
for i in range(len(campaign_params['character_names'])):
    print(textwrap.fill(f"{campaign_params['character_names'][i]}: {campaign_params['character_personalities'][i]}", width=170))
    print('\n')
    print(campaign_params['character_sheets'][i])
    print('\n')

print('Player Personalities')
for i in range(len(campaign_params['player_names'])):
    print(
        textwrap.fill( 
            f"{campaign_params['player_names'][i]}: {campaign_params['player_personalities'][i]}",
            width=170))
    print('\n')


📊 Extracting campaign parameters...
Here are the character sheets based on the available information:

Jinx:
Level: 1
Class: Paladin
Race: Mountain Dwarf
Background: Unknown
Alignment: Chaotic Good
Strength: 15
Dexterity: 10
Constitution: 14
Intelligence: 8
Wisdom: 12
Charisma: 13
Hit Points: 12/12
Armor Class: 18
Proficiency Bonus: +2
Saving Throw Proficiencies: Wisdom, Charisma
Skill Proficiencies: Unknown
Languages: Common, Dwarvish
Equipment: Chainmail armor, maul, shield
Spells Known: Unknown
Special Abilities: Darkvision, Dwarven Resilience, Stonecunning
Notes: Worships Ximesi, sage of mountains deity of light and fire. From a clan in the Ximesi Mountains. Part of a family tradition of guardians and warriors.

Faen:
Level: 1
Class: Warlock (The Fiend patron)
Race: Moon Elf
Background: Noble
Alignment: Neutral Good
Strength: 8
Dexterity: 14
Constitution: 12
Intelligence: 13
Wisdom: 10
Charisma: 15
Hit Points: 9/9
Armor Class: 13
Proficiency Bonus: +2
Saving Throw Proficiencies: Wi

In [3]:
campaign_params['initial_scenario']['1']['date']

'2025-08-11T13:52:48.459614'

## Step 2: Create Characters

Now let's create our AI characters using the player count from the human campaign.

In [4]:
# Create characters based on the campaign parameters
print("🧙 Creating characters...")

characters = sim.create_characters(campaign_params, model=model)

print(f"\n✨ Created {len(characters)} characters:")
for char in characters:
    print(f"  🗡️  {char.name} ({char.dnd_class}): {char.personality}")
    print(f"         {char.character_sheet}")

# Get character names for turn sampling
character_names = [char.name for char in characters]
print(f"\n👥 Character roster: {character_names}")

🧙 Creating characters...

✨ Created 5 characters:
  🗡️  Dungeon Master (None): None
         (None,)
  🗡️  Jinx (druid): A fiery and confident young female dwarf paladin from the Ximesi Mountains. Jinx is devoutly religious, serving Ximesi the deity of Light and Fire. She comes from a long line of female warriors and guardians in her clan. Jinx is eager to prove herself and fight evil, sometimes to the point of recklessness. She has a strong sense of justice and duty, and little patience for those she sees as unworthy or dishonorable. Despite her serious demeanor, Jinx shows flashes of humor and camaraderie with her fellow guardians.
         ({'Level': 1, 'Class': 'Paladin', 'Race': 'Mountain Dwarf', 'Background': 'Unknown', 'Alignment': 'Chaotic Good', 'Strength': 15, 'Dexterity': 10, 'Constitution': 14, 'Intelligence': 8, 'Wisdom': 12, 'Charisma': 13, 'Hit Points': '12/12', 'Armor Class': 18, 'Proficiency Bonus': '+2', 'Saving Throw Proficiencies': ['Wisdom', 'Charisma'], 'Skill Pro

## Step 3: Generate Turn Sequence

Create a realistic turn sequence using uniform sampling across all characters.

In [5]:
# Generate a turn sequence for our mini-session
total_turns = 20#len(campaign_params['character_turns'])-1  # Adjust for desired session length
from_human_game = True

if from_human_game:
    turn_sequence = np.array(campaign_params['character_turns'])[1:total_turns] # start at 1 to remove 1st DM post
else:
    turn_sequence = sim.sample_turn_sequence(
    character_names=character_names,
    total_turns=total_turns,
    method='uniform')

#print(f"\n📋 Turn sequence: {turn_sequence}")

# Show turn distributions
print('Total turn number: ' + str(total_turns))
turn_counts = Counter(turn_sequence)
print(f"\n📊 Turn distribution:")
for char_name, count in turn_counts.items():
    print(f"  {char_name}: {count} turns ({count/total_turns*100:.1f}%)")

Total turn number: 20

📊 Turn distribution:
  Jinx: 3 turns (15.0%)
  Faen: 4 turns (20.0%)
  Dmitrei: 3 turns (15.0%)
  Thokk: 5 turns (25.0%)
  Dungeon Master: 4 turns (20.0%)


## Step 4: Initialize Game Session

Create the game session manager that will orchestrate our D&D simulation.

In [6]:
# Create game session
print("🎭 Initializing game session...")
game_session = sim.GameSession(characters=characters)

print(f"✅ Game session created with {len(characters)} characters")
print(f"📝 Game log initialized (currently {len(game_session.game_log)} events)")

🎭 Initializing game session...
✅ Game session created with 5 characters
📝 Game log initialized (currently 0 events)


## Step 6: Run the Simulation

Now let's run the complete D&D simulation and watch our AI characters interact!

In [7]:
# Run the simulation
print("🚀 Starting D&D simulation...")
print("\n" + "="*60)

# Execute the scenario
game_session.run_scenario(
    initial_scenario=campaign_params['initial_scenario'],
    turn_sequence=turn_sequence,
    include_player_personalities=False,
    print_cache=True)

🚀 Starting D&D simulation...

=== INITIAL GAME LOG ===
{'1': {'date': '2025-08-11T13:52:48.459614', 'player': 'LightSpeed', 'character': 'Dungeon Master', 'in_combat': False, 'paragraphs': {'0': {'text': 'Guardians of Gridori', 'actions': ['name_mention: gridori'], 'label': 'in-character'}, '1': {'text': '[Recruitment Closed]', 'actions': [], 'label': 'in-character'}, '2': {'text': "General Information : A PbP 5E D&D campaign for 3-4 players, new players and veterans are welcome. New characters begin at first level. Race and class options are restricted only to the  Player's Handbook  (dragonborn are not available for this campaign). Ability scores can be determined by using standard array or point buy. Advancement will be achieved through milestones. If you are interested, please post a character concept including  name ,  race ,  class , and  background . Furthermore, please include a  brief  answer to the question:  Why are you a guardian?  Please consider the information in the  Ca

## Export Results

Save the simulation results for further analysis.

In [8]:
# Export simulation results

# Save to file

output_file = f"llm_campaign_{campaign_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
output_path = os.path.join(os.getcwd(), '..', 'data', 'llm-games', output_file)
with open(output_path, 'w') as f:
    json.dump(game_session.game_log, f, indent=2)

print(f"💾 Results saved to: {output_path}")
print(f"📁 File size: {os.path.getsize(output_path)} bytes")

💾 Results saved to: /Users/annie/Code/Repositories/dnd-dynamics/tutorials/../data/llm-games/llm_campaign_10391-guardians-of-gridori_20250811_135447.json
📁 File size: 38315 bytes
